In [ ]:
import pandas as pd
import import_ipynb
import sys
import importlib
import csv

from YOLO import YOLO
from Facial_Detection import FD
from Args import imageCalificationArguments
from Data_Utilities import load_csv
from Image_Utilities import readImage, showImage, showImageBox, getImageCrop

In [ ]:
def clasifyImages(dataPath, imagePath, boxesPath, RP, yolo, startingImage=None):
    """
    This method utilize YOLO to detect persons and extract it boxes
    """
    print(dataPath)
    print(imagePath)
    print(RP)
    
    csv = load_csv(dataPath)
    headers = csv[0]
    data = csv[1:]
    
    df = pd.DataFrame(data, columns = headers) 
    rpRows = df.loc[df['RP'] == RP]
    
    
    imageTotal = 1
    booleanCheck = True
    for index, row in rpRows.iterrows():
        if startingImage is not None and booleanCheck:
            if startingImage != row["name"]:
                continue
            else:
                booleanCheck = False
        if imageTotal > 100:
            print("nextImage:" + row["name"])
            break
        print("IMAGE Nº:",imageTotal)
        print(imagePath+row["name"])
        print(row["numbers"])
        answer = ""
        yolo.setImage(imagePath+row["name"])
        yolo.process_image()
        yolo.detect()
        yolo.showDetections(row["name"],boxesPath, RP, row["numbers"].split())
        imageTotal = imageTotal+1
    
def showBoxesImages(dataPath, imagePath, RP=None, startingImage=None):
    """
    This method shows the boxes and runner bib on the given image
    """
    
    csv = load_csv(dataPath)
    headers = csv[0]
    data = csv[1:]
    
    df = pd.DataFrame(data, columns = headers)
    
    if RP != None:
        df = df.loc[df['RP'] == RP]
    
    print(headers)
    currentRP = None
    imageTotal = 1
    booleanCheck = True
    for index, row in df.iterrows():
        if startingImage is not None and booleanCheck:
            if row["Image"] != startingImage:
                continue
            else:
                booleanCheck = False
        if imageTotal > 5:
            print("nextImage:" + row["Image"])
            break
        print("IMAGE Nº:",imageTotal)
        if currentRP == None or currentRP != row["RP"]:
            currentRP = row["RP"]
        answer = ""
        print(int(row["X"]),int(row["Y"]),int(row["Width"]),int(row["Height"]))
        showImage(getImageCrop(readImage(imagePath+row["Image"]),[int(row["X"]),int(row["Y"]),int(row["Width"]),int(row["Height"])]),row["Dorsal"])
        answer = input("Next?(enter/q to exit)")
        if answer == "q":
            break
        imageTotal = imageTotal+1
    
def clasifyFaces(bodyCSVPath, emotionsCSVPath, imagePath, RP, fd, startingImage=None, dorsal=None):
    """
    This method utilize FD to get the emotions of a given body box and save it
    """
    
    csvData = load_csv(bodyCSVPath)
    headers = csvData[0]
    data = csvData[1:]
    
    df = pd.DataFrame(data, columns = headers)
    
    if RP != None:
        df = df.loc[df['RP'] == RP]
    
    print(df)
    imageTotal = 1
    booleanCheck = True
    dataMatrix = []
    for index, row in df.iterrows():
        if startingImage is not None and booleanCheck:
            if row["Image"] != startingImage:
                continue
            elif int(row["Dorsal"]) != dorsal:
                continue
            else:
                booleanCheck = False

        if imageTotal > 10000:
            print("nextImage: \"",row["Image"],"\", ", row["Dorsal"], sep="")
            break
        print("IMAGE Nº:",imageTotal,"\n\033[1;32m\"",row["Image"],"\", ", row["Dorsal"],"\033[00m", sep="")
        coords = [int(row["X"]),int(row["Y"]),int(row["Width"]),int(row["Height"])]
        fd.setImage(imagePath+row["Image"])
        fd.setCoords([coords])
        fd.facialDetection()
        fd.checkImages()
        averageEmotions, booleanNext = fd.perfornmDetection()
        print("AVERAGE EMOTIONS: ",averageEmotions)
        imageTotal = imageTotal+1
        if not booleanNext:
            print("GOING NEXT")
        else:
            print("DATA TO SAVE:")
            toSave = (row["Dorsal"], RP, averageEmotions["anger"], averageEmotions["disgust"], 
                      averageEmotions["fear"], averageEmotions["happiness"], averageEmotions["sadness"],
                      averageEmotions["surprise"], averageEmotions["neutral"])
            print(toSave)
            dataMatrix.append(toSave)
    print("\n")
    for dm in dataMatrix:
        print(dm)
    
    performAverageEmotionsSave(dataMatrix, emotionsCSVPath)
    

def performAverageEmotionsSave(data, savePath):
    """
    This method perform the average emotion total for every runner bib
    """
    while data != []:
        dorsal = None
        cummulative = [0,0,0,0,0,0,0]
        dataToRemove = []
        total = 0
        for d in data:
            if dorsal == None:
                dorsal = d[0]
                RP = d[1]
                
            
            if d[0] == dorsal:
                
                cummulative = [sum(x) for x in zip(cummulative, d[2:])]
                total = total+1
                dataToRemove.append(d)
        
        for dtr in dataToRemove:
            data.remove(dtr)
        cummulative = [c/total for c in cummulative]
        toSave = (dorsal, RP, cummulative[0], cummulative[1], cummulative[2], cummulative[3], 
                  cummulative[4], cummulative[5], cummulative[6])
        print("\nRunner bib:", dorsal)
        print(toSave)
        #print(sum(toSave[2:]))
        with open(savePath, "a") as f:
            writer = csv.writer(f)
            writer.writerow(toSave)
            
            
    
    

In [ ]:
def imageCalificationMain(args, yoloArgs, fdArgs):
    """
    This method manages the main pipeline for emotion detection. It utilize the YOLO an FD classes
    """
        
    answer = ""
    answer = input("Do you want to execute person box detection(YOLO)?(y/n)")
    if answer == "y":
        while not answer.isnumeric() and answer != "q":
            answer = input("What RP do you want to clasify?(0-9,11,13, q for quit)")

        if answer == "q":
            sys.exit()

        yolo = YOLO(yoloArgs)
        clasifyImages(args["dataFolder"]+args["sortedDataCsvName"], args["imageFolder"],
                       args["dataFolder"]+args["boxesDataCsvName"],"RP"+answer, yolo)
    
    answer = input("Do you want to see person boxes?(y/n)")
    if answer == "y":
        while not answer.isnumeric():
            answer = input("Of what RP?(0-13)(q for quit)")
            #answer = ""
        showBoxesImages(args["dataFolder"]+args["boxesDataCsvName"], args["imageFolder"], "RP"+answer)
    
    
    answer = ""
    answer = input("Do you want to execute emotion detection?)?(y/n)")
    if answer == "y":
        while not answer.isnumeric() and answer != "q":
            answer = input("From what RP do you want the emotions of?(0-9,11,13)(q for quit)")

        if answer == "q":
            sys.exit()
        
        fd = FD(fdArgs)
        fd.initDetector()
        clasifyFaces(args["dataFolder"]+args["boxesDataCsvName"], 
                     args["dataFolder"]+args["emotionsDataCsvName"],
                     args["imageFolder"], "RP"+answer, fd)

In [ ]:
#from Args import preWorkArguments, yoloArguments, fdArguments, imageCalificationArguments, statisticsExtractionsArguments
#
#imageCalificationMain(imageCalificationArguments(), yoloArguments(), fdArguments())